# Búsquedas semánticas con películas de Star Wars

## Entorno:

```shellscript
docker run -p 6333:6333 -p 6334:6334 -v $(pwd)/q_storage:/qdrant/storage:z qdrant/qdrant
```

```shellscript	
python -m venv .venv
source .venv/bin/activate
```

Dataset: https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots


In [14]:
%pip install -r requirements.txt
#!jupyter nbextension enable --py widgetsnbextension


Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_json('datasets/star_wars_plots.json')
print('Columns:', df.columns, ', Records:', df.shape[0])
df.head()
startwars_movies = df

Columns: Index(['title', 'plot'], dtype='object') , Records: 11


### Creamos embeddings del argumento de las películas

In [16]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

client = openai.Client()
# New embeddings: https://openai.com/blog/new-embedding-models-and-api-updates
# Embedding guide: https://platform.openai.com/docs/guides/embeddings
# Pricing: https://openai.com/pricing#language-models
EMBEDDING_MODEL = "text-embedding-3-small"

text_sample = "Hace mucho tiempo en una galaxia muy, muy lejana ..."
response = client.embeddings.create(model=EMBEDDING_MODEL, input=text_sample)
vector = response.data[0].embedding
print('Dimension:', len(vector), vector[:2], '...', vector[-2:])



Dimension: 1536 [0.014705708250403404, -0.053972844034433365] ... [0.04716505482792854, -0.02482839673757553]


### Preparamos langchain para hacer búsquedas semánticas

In [17]:
# LangChain Doc: https://python.langchain.com/docs/get_started/introduction
from langchain.embeddings.openai import OpenAIEmbeddings

oai_embedding = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Embedding texto: oaie.embed_query(text_sample)

r = oai_embedding.embed_documents([text_sample, text_sample], 16)
MODEL_DIMENSION = len(r[0])
print('Vectors:', len(r), 'Dimension:', MODEL_DIMENSION, r[0][:2], '...', r[0][-2:])




Vectors: 2 Dimension: 1536 [0.014705707666277976, -0.05397284189057806] ... [0.047165052954485434, -0.024828395751366802]


In [39]:
from langchain_community.docstore.document import Document
CHUNK_WORDS = 200 # OpenAI model soporta hasta 8000 tokens (aprox)

# Creamos chunks de texto de unas 170 palabras con un solape de 10 palabras
def chunk_text(text, chunk_size=CHUNK_WORDS, overlap=CHUNK_WORDS//20):
    chunks = []
    words = text.split()
    for i in range(0, len(words), chunk_size-overlap):
        chunks.append(' '.join(words[i:i+chunk_size]))
    return chunks

def create_docs_from_df(df, text_column='plot', chunk_size=CHUNK_WORDS):
    docs = []
    for i, row in df.iterrows():
        plot_chunks = chunk_text(row[text_column], chunk_size)
        for chunk in plot_chunks:
            metadata = dict(title=row['title'])
            metadata['plot_chunk'] = chunk
            doc = Document(page_content=chunk, metadata=metadata)
            docs.append(doc)

        print(i, row['title'], ' => chunks:', len(plot_chunks))

    print('\nTotal documents:', len(docs))
    for d in docs[:4]:
        print('🎞️ ➡️', d.metadata['title'], '📄 =>', d.page_content[:100])
    return docs

documents = create_docs_from_df(startwars_movies)


0 Star Wars: Episodio IV - Una nueva esperanza  => chunks: 8
1 Star Wars: Episodio V - El Imperio contraataca  => chunks: 12
2 Star Wars: Episode VI - Return of the Jedi  => chunks: 17
3 Star Wars: Episodio I - La amenaza fantasma  => chunks: 5
4 Star Wars: Episodio II - El ataque de los clones  => chunks: 13
5 Star Wars: Episodio III - La venganza de los Sith  => chunks: 28
6 Star Wars: Episodio VII - El despertar de la Fuerza  => chunks: 20
7 Star Wars: Episodio VIII - Los últimos Jedi  => chunks: 16
8 Star Wars: Episodio IX - El ascenso de Skywalker  => chunks: 17
9 Rogue One: una historia de Star Wars  => chunks: 20
10 Han Solo: una historia de Star Wars  => chunks: 6

Total documents: 162
🎞️ ➡️ Star Wars: Episodio IV - Una nueva esperanza 📄 => Hace mucho tiempo en una galaxia muy, muy lejana ... Son tiempos de guerra civil. Naves rebeldes han
🎞️ ➡️ Star Wars: Episodio IV - Una nueva esperanza 📄 => Darth Vader a la cabeza. Durante el asalto, capturan a la princesa, quien antes logr

### Conectamos con Qdrant para crear una colección y alimentarla con los embeddings


In [19]:
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

MOVIES_COLLECTION = "movies_sw_openai"


client = QdrantClient(host="localhost", prefer_grpc=True)
client.recreate_collection(MOVIES_COLLECTION, VectorParams(size=MODEL_DIMENSION, distance=Distance.COSINE))

# Qdrant store: https://python.langchain.com/docs/integrations/vectorstores/qdrant
qdrant = Qdrant(
    client=client,
    collection_name=MOVIES_COLLECTION,
    embeddings=oai_embedding
)
print('Collections', qdrant.client.get_collections())



Collections collections=[CollectionDescription(name='movies_sw'), CollectionDescription(name='movies_sw_openai')]


In [20]:
op = qdrant.add_documents(documents)

print('IDs:', op[:2], '...', op[-2:])
print('Collections:', client.get_collections().collections)
print('Collection:', MOVIES_COLLECTION, 'rows:', client.get_collection(MOVIES_COLLECTION))



IDs: ['ebbc1f954b054d6eb4b1ba29a36dc554', 'a2deb8af583942d9be841bf3748fc9b2'] ... ['a35818d8bbd840efade4a2caa9715925', 'e16cb3df6f97427ebc7b14865776b2c2']
Collections: [CollectionDescription(name='movies_sw'), CollectionDescription(name='movies_sw_openai')]
Collection: movies_sw_openai rows: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=162 indexed_vectors_count=0 points_count=162 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_

### Realizamos búsquedas semánticas sobre la saga Star Wars

In [21]:
queries = [
    'Anakin gana una carrera cuando era un niño',
    'Los clones reciben la orden de ejecutar a los Jedi',
    'Han Solo gana el Halcón Milenario en una partida de cartas',
    'Kylo Ren mata a su padre, Han Solo',
    'Palpatine es derrotado definitivamente por Rey',
    # Less accurate queries
    'Luke descubre que Darth Vader es su padre',
    'Luke encuentra a Yoda y es entrenado como Jedi',
]

for q in queries:    
    results = qdrant.similarity_search_with_score(q, k=3)    
    print('Q:', q)
    for doc, score in results:
        print('     ', score, ' => ', doc.metadata['title'])
        print('     ', f'(id: {doc.metadata["_id"]})', doc.page_content)



Q: Anakin gana una carrera cuando era un niño
      0.6705865859985352  =>  Star Wars: Episodio I - La amenaza fantasma
      (id: 7f5614fe-f8d9-4312-bd6f-fc77264e7418) el niño gana la carrera, se quedarán con los repuestos para la nave y con la libertad de Anakin. Watto acepta, al creer que el mayor oponente de Anakin, Sebulba, le ganaría.Ya en la carrera, Anakin tiene problemas ya que Sebulba había roto un elemento de su pod. Sin embargo, el niño se las ingenia para estabilizar la nave y finalmente gana la carrera. Luego de una emotiva despedida entre Shmi y Anakin, todos se dirigen hacia la nave de la Reina para trasladarse hacia Coruscant, capital de la República. Antes de llegar a la nave, Qui-Gon es sorprendido por el aprendiz de Lord Sidious, Darth Maul, y mantienen una pequeña lucha con sus sables. Finalmente, Qui-Gon logra huir en la nave. Una vez en Coruscant, Qui-Gon y Obi-Wan se dirigen hacia el Consejo Jedi para proponer el entrenamiento de Anakin, pero es rechazado debido

### Creamos un Chat Agent experto en Star Wars con langchain

In [77]:
MY_MOVIE_COLL = "my_movie"
qdrant_chat = Qdrant(
    client=client,
    collection_name=MY_MOVIE_COLL,
    embeddings=oai_embedding
)
client.recreate_collection(MY_MOVIE_COLL, VectorParams(size=MODEL_DIMENSION, distance=Distance.COSINE))

my_movie = pd.read_json('datasets/rey_legacy_plot.json')
print('Columns:', my_movie.columns, ', Records:', my_movie.shape[0])
my_movie.head()

docs = create_docs_from_df(my_movie)
qdrant_chat.add_documents(docs)

# http://localhost:6333/dashboard/#/collections/my_movie
# http://localhost:6333/collections/my_movie

Columns: Index(['title', 'plot'], dtype='object') , Records: 1
0 Star Wars: Episodio X - Rey Legacy  => chunks: 5

Total documents: 5
🎞️ ➡️ Star Wars: Episodio X - Rey Legacy 📄 => Hace mucho tiempo en una galaxia muy, muy lejana ... Después de haber derrotado al temible Palpatine
🎞️ ➡️ Star Wars: Episodio X - Rey Legacy 📄 => tiene que actuar rápido y sin dudarlo se infiltra en la nave 'Korriban Avenger' para rescatar a los 
🎞️ ➡️ Star Wars: Episodio X - Rey Legacy 📄 => Krayer, sin poner en riesgo a su nuevo amigo, Aron acepta y se adentra en la nave. La pelea con Kray
🎞️ ➡️ Star Wars: Episodio X - Rey Legacy 📄 => Aron Tano con el dispositivo de localización. Rey y Aron se hacen con el control de la nave, y detie


['2c43a9c657954f409f2c75b7008bacf5',
 '122c4ae257be4423a2bb11b509ef4f80',
 'f8dc819d71844fa2be1ffddf52a9cf69',
 '2c114cb75c6749c39dfc4ce74274eb5d',
 '41ed88d4bd004c3f89db51933a8ac01f']

In [78]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

#OpenAI LLM
# OpenAI models: https://platform.openai.com/docs/models
# OpenAI pricing: https://openai.com/pricing#language-models
llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"),
                 model_name="gpt-3.5-turbo-0125", # gpt-4-turbo-preview
                 temperature=0.0)

# Conversation memory
#conv_mem = ConversationBufferWindowMemory(memory_key='history', k=5, return_messages=True)

qa = RetrievalQA.from_llm(
    llm=llm,
    retriever=qdrant_chat.as_retriever()
)


In [79]:
def print_qa(q):
    print('\nQ:', q)
    answer = qa.run(q)
    print('A:', answer)

#print_qa("¿quién es el padre de Luke Skywalker?")
#print_qa("¿A qué se dedica Rey en Tatooine?")
#print_qa("¿Quién avisa a Rey para que viaje a Mercuroon?")
#print_qa("¿Qué se encuentra Rey al llegar a Mercuroon por primera vez?")
#print_qa("¿Cómo se llama el lider de la nueva orden Sith?")
#print_qa("¿Cómo se llama la nave de Darth Krayer?")
print_qa("¿A quién encuentra Rey en la nave 'Korriban Avenger'?")
#print_qa("¿Cómo derrota Rey a Krayer, quién la ayuda?")
#print_qa("¿quién es Aron?")
#print_qa("¿cómo ayuda Aron en la nave de Krayer a Rey?")
#print_qa("¿Qué decide hacer Rey en el templo de Mercuroon?")
#print_qa("¿Quién acompaña a Rey a Mercuroon?")
#print_qa("¿A quien pide ayuda Rey después de volver a Mercuroon?")

## Preguntas sin contexto
#print_qa("¿Quién es el padre de Rey?")
print_qa("¿quién es la madre de Aron?")



Q: ¿A quién encuentra Rey en la nave 'Korriban Avenger'?
A: Rey encuentra a un grupo de jóvenes secuestrados en la nave 'Korriban Avenger', uno de ellos es un joven adolescente llamado 'Aron Tano', quien resulta ser el nieto de 'Ahsoka Tano'.

Q: ¿quién es la madre de Aron?
A: No hay información proporcionada en el contexto sobre quién es la madre de Aron.


In [80]:
my_plot = '''Aron descubre en la nave de Ahsoka que sus padres viven, su padre es Lando Tano y su madre Fenny Shuuka, 
pero están prisioneros de unas minas de cristales en el planeta Kreeltor. Aron decide ir a rescatarlos y Rey le acompaña.
'''
new_doc = Document(page_content=my_plot, metadata=dict(title="Ahsoka Legacy"))

qdrant_chat.add_documents([new_doc])

print_qa("¿quién es la madre de Aron?")
print_qa("¿dónde se encuentran los padres de Aron?")





Q: ¿quién es la madre de Aron?
A: La madre de Aron es Fenny Shuuka.

Q: ¿dónde se encuentran los padres de Aron?
A: Los padres de Aron, Lando Tano y Fenny Shuuka, se encuentran prisioneros en unas minas de cristales en el planeta Kreeltor.
